<a href="https://colab.research.google.com/github/friedelj/ML540/blob/main/JFriedel_USD540_Assignment3_1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JFriedel                                                     Assignment 3_1                                                     6-2-25

In [ ]:
import boto3
import sagemaker

original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

In [ ]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

In [ ]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-asmt3"

print(default_s3_bucket_name)

In [ ]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io

s3_client = boto3.client("s3", region_name=region)

fraud_detection_bucket_name = f"sagemaker-example-files-prod-{region}"
identity_file_key = ("housing_gmaps_data_raw.csv")
transaction_file_key = ("housing.csv")

# ------------DATA CLEANING

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from sklearn.neighbors import NearestNeighbors
import re
from datetime import datetime
import time

In [ ]:
# Load the CSV file
df_housing = pd.read_csv('housing.csv')

# Load the CSV file
df_gmaps = pd.read_csv('housing_gmaps_data_raw.csv')

In [ ]:
# Merge df_housing with df_gmaps on 'longitude' and 'latitude'
df_housing = df_housing.merge(
    df_gmaps[['longitude', 'latitude', 'postal_code']],
    on=['longitude', 'latitude'],
    how='left'
)

# Save the updated df_housing to a CSV file
df_housing.to_csv('housing_with_postal_code.csv', index=False)

In [ ]:
# Display total missing values per column
missing_per_column = df_housing.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_housing.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
# Compute the rounded average total_bedrooms for each postal_code
avg_bedrooms_by_postal = df_housing.groupby('postal_code')['total_bedrooms'].mean().round()

# Define a function to apply the group average to missing values
def fill_bedrooms(row):
    if pd.isna(row['total_bedrooms']):
        return avg_bedrooms_by_postal.get(row['postal_code'], np.nan)
    else:
        return row['total_bedrooms']

# Apply the function
df_housing['total_bedrooms'] = df_housing.apply(fill_bedrooms, axis=1)

# Save the modified DataFrame to CSV
df_housing.to_csv('housing_bedrooms_filled.csv', index=False)

In [ ]:
# Display total missing values per column
missing_per_column = df_housing.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_housing.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
# Separate known and unknown postal_code rows
known = df_housing[df_housing['postal_code'].notna()].copy()
unknown = df_housing[df_housing['postal_code'].isna()].copy()

# Build a KDTree from known locations
tree = cKDTree(known[['latitude', 'longitude']])

# Query the closest known point for each unknown point
distances, indices = tree.query(unknown[['latitude', 'longitude']], k=1)

# Assign the closest known postal_code to the unknown rows
closest_postal_codes = known.iloc[indices]['postal_code'].values
df_housing.loc[unknown.index, 'postal_code'] = closest_postal_codes

# Save the updated DataFrame to CSV
df_housing.to_csv('housing_postal_filled.csv', index=False)

In [ ]:
# Display total missing values per column
missing_per_column = df_housing.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_housing.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
# Add new column by dividing total_bedrooms by households and rounding the result
df_housing['bedrooms_per_household'] = (df_housing['total_bedrooms'] / df_housing['households']).round()

# Save the updated DataFrame to CSV
df_housing.to_csv('housing_bedrooms_per_household.csv', index=False)

In [ ]:
df_housing.head(10)

In [ ]:
# Display total missing values per column
missing_per_column = df_housing.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_housing.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
# Display total missing values per column
missing_per_column = df_gmaps.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_gmaps.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
# Replace empty strings with NaN (if needed)
df_gmaps['administrative_area_level_1-political'].replace('', pd.NA, inplace=True)

# Fill missing values with 'California'
df_gmaps['administrative_area_level_1-political'].fillna('California', inplace=True)

# Save the modified DataFrame to a CSV file
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Replace empty strings with NaN (if needed)
df_gmaps['postal_code_suffix'].replace('', pd.NA, inplace=True)

# Fill missing values with 9999 and convert the column to integer
df_gmaps['postal_code_suffix'] = df_gmaps['postal_code_suffix'].fillna(9999).astype(int)

# Save the modified DataFrame to a CSV file
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Replace empty strings with NaN (if needed)
df_gmaps['street_number'].replace('', pd.NA, inplace=True)

# Fill missing values with "0" (as a string to match text-based route values)
df_gmaps['street_number'] = df_gmaps['street_number'].fillna('0')

# Save the modified DataFrame to a CSV file
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Separate rows with and without the target value
df_missing = df_gmaps[df_gmaps['administrative_area_level_2-political'].isna()]
df_known = df_gmaps[df_gmaps['administrative_area_level_2-political'].notna()]

# Fit NearestNeighbors on known locations
nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn.fit(df_known[['latitude', 'longitude']])

# Find closest matches for missing rows
distances, indices = nn.kneighbors(df_missing[['latitude', 'longitude']])

# Fill missing values
df_gmaps.loc[df_missing.index, 'administrative_area_level_2-political'] = \
    df_known.iloc[indices.flatten()]['administrative_area_level_2-political'].values

# Save the modified DataFrame to a CSV
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Function to extract the route from the address
def extract_route(address):
    if pd.isna(address):
        return None
    match = re.search(r'\d+\s+([^,]+)', address)
    return match.group(1).strip() if match else None

# Replace empty strings with NaN if necessary
df_gmaps['route'].replace('', pd.NA, inplace=True)

# Fill missing 'route' values
df_gmaps['route'] = df_gmaps.apply(
    lambda row: extract_route(row['address']) if pd.isna(row['route']) else row['route'],
    axis=1
)

# Save the modified DataFrame to a CSV file
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Replace empty strings with NaN (if applicable)
df_gmaps['route'].replace('', pd.NA, inplace=True)

# Fill missing values in 'route' with the default string
df_gmaps['route'] = df_gmaps['route'].fillna('strret name not filled in')

# Save the modified DataFrame to a CSV file
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Convert empty strings to NaN
df_gmaps['locality-political'].replace('', np.nan, inplace=True)

# Identify rows with missing and non-missing 'locality-political'
df_missing = df_gmaps[df_gmaps['locality-political'].isna()]
df_known = df_gmaps[df_gmaps['locality-political'].notna()]

# Use NearestNeighbors to find closest known point
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(df_known[['latitude', 'longitude']])
distances, indices = nbrs.kneighbors(df_missing[['latitude', 'longitude']])

# Fill missing 'locality-political' with nearest neighbor's value
for i, idx in enumerate(df_missing.index):
    nearest_index = df_known.index[indices[i][0]]
    df_gmaps.at[idx, 'locality-political'] = df_known.at[nearest_index, 'locality-political']

# Save the modified DataFrame
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Convert empty strings to NaN
df_gmaps['postal_code'].replace('', np.nan, inplace=True)

# Split data into rows with missing and non-missing postal_code
df_missing = df_gmaps[df_gmaps['postal_code'].isna()]
df_known = df_gmaps[df_gmaps['postal_code'].notna()]

# Fit NearestNeighbors using non-missing data
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(df_known[['latitude', 'longitude']])
distances, indices = nbrs.kneighbors(df_missing[['latitude', 'longitude']])

# Fill missing postal_code values from nearest neighbor
for i, idx in enumerate(df_missing.index):
    nearest_idx = df_known.index[indices[i][0]]
    df_gmaps.at[idx, 'postal_code'] = df_known.at[nearest_idx, 'postal_code']

# Save the updated DataFrame
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Convert empty strings to NaN
df_gmaps['neighborhood-political'].replace('', np.nan, inplace=True)

# Split the DataFrame into rows with and without missing 'neighborhood-political'
df_missing = df_gmaps[df_gmaps['neighborhood-political'].isna()]
df_known = df_gmaps[df_gmaps['neighborhood-political'].notna()]

# Fit NearestNeighbors model on known values
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(df_known[['latitude', 'longitude']])
distances, indices = nbrs.kneighbors(df_missing[['latitude', 'longitude']])

# Fill in missing 'neighborhood-political' values
for i, idx in enumerate(df_missing.index):
    nearest_idx = df_known.index[indices[i][0]]
    df_gmaps.at[idx, 'neighborhood-political'] = df_known.at[nearest_idx, 'neighborhood-political']

# Save the updated DataFrame
df_gmaps.to_csv('df_gmaps_filled.csv', index=False)

In [ ]:
# Display total missing values per column
missing_per_column = df_gmaps.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_gmaps.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
# Keep only the first 12 columns
df_gmaps = df_gmaps.iloc[:, :12]

# Save the result to a new CSV
df_gmaps.to_csv('housing_gmaps_data_trimmed.csv', index=False)

# Display the result
print(df_gmaps.head())

In [ ]:
# Display total missing values per column
missing_per_column = df_gmaps.isnull().sum()
print("Missing values per column:\n", missing_per_column)

# Check if any value is missing in the entire DataFrame
any_missing = df_gmaps.isnull().values.any()
print("\nIs there any missing data in the file?:", any_missing)

In [ ]:
df_gmaps.head(10)

In [ ]:
# Extract unique values (including NaNs if needed) and create the new DataFrame
df_neighborhood = pd.DataFrame({
    'neighborhood': df_gmaps['neighborhood-political'].dropna().unique()
})

# Sort for readability
df_neighborhood = df_neighborhood.sort_values(by='neighborhood').reset_index(drop=True)

# Display the first 10 rows
print(df_neighborhood.head(10))

# Save to CSV
df_neighborhood.to_csv('neighborhood.csv', index=False)

In [ ]:
# Prepare coordinate data
gmaps_coords = df_gmaps[['latitude', 'longitude']].values
housing_coords = df_housing[['latitude', 'longitude']].values

# Fit NearestNeighbors model on df_gmaps
nn_model = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn_model.fit(gmaps_coords)

# Find the index of the closest neighbor in df_gmaps for each row in df_housing
distances, indices = nn_model.kneighbors(housing_coords)

# Get the corresponding neighborhood values from df_gmaps
matched_neighborhoods = df_gmaps.iloc[indices.flatten()]['neighborhood-political'].values

# Add the matched neighborhoods to df_housing
df_housing['neighborhood'] = matched_neighborhoods

# Save to CSV
df_housing.to_csv('df_housing_appended.csv', index=False)

In [ ]:
# Group df_housing by 'neighborhood' and calculate the average of 'median_house_age'
average_house_age = df_housing.groupby('neighborhood')['housing_median_age'].mean().reset_index()

# Rename the column to match the target column name
average_house_age.columns = ['neighborhood', 'median-house-age']

# Merge the average values into df_neighborhood
df_neighborhood = df_neighborhood.merge(average_house_age, on='neighborhood', how='left')

# Save the updated DataFrame to a CSV file
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Get current PC time
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Add 'event_time' column with current time for each row
df_neighborhood['event_time'] = current_time

# Save to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Define a function to convert a number to a decade range string
def convert_to_age_range(age):
    if pd.isna(age):
        return None  # or a default value like "Unknown"
    lower = int(age) // 10 * 10
    upper = lower + 9
    return f"{lower} to {upper} years"

# Apply the function to the 'median-house-age' column
df_neighborhood['median-house-age'] = df_neighborhood['median-house-age'].apply(convert_to_age_range)

# Save the modified DataFrame to a CSV file
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Group df_housing by 'neighborhood' and compute the average of 'total_households'
average_total_households = df_housing.groupby('neighborhood')['households'].mean().reset_index()

# Rename the column for merging
average_total_households.columns = ['neighborhood', 'total-households']

# Merge the average values into df_neighborhood
df_neighborhood = df_neighborhood.merge(average_total_households, on='neighborhood', how='left')

# Save the updated DataFrame to a CSV file
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Calculate average median_house_value per neighborhood, capped at 500000
avg_median_house_value = (
    df_housing.groupby('neighborhood')['median_house_value']
    .mean()
    .clip(upper=500000)  # Cap at 500,000
    .reset_index()
)

# Rename column to match the target in df_neighborhood
avg_median_house_value.columns = ['neighborhood', 'median-house-value']

# Merge this result into df_neighborhood
df_neighborhood = df_neighborhood.merge(avg_median_house_value, on='neighborhood', how='left')

# Save the modified DataFrame to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Round the 'medium-house-value' column to 0 decimal places and convert to int
df_neighborhood['median-house-value'] = df_neighborhood['median-house-value'].round(0).astype('Int64')

# Save the modified DataFrame to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Group df_housing by 'neighborhood' and compute the average of 'total-households'
average_total_households = df_housing.groupby('neighborhood')['households'].mean().reset_index()

# Rename the column for merging
average_total_households.columns = ['neighborhood', 'total-households']

# Merge the average values into df_neighborhood
df_neighborhood = df_neighborhood.merge(average_total_households, on='neighborhood', how='left')

# Save the updated DataFrame to a CSV file
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Compute average bedrooms_per_household per neighborhood in df_housing
avg_bedrooms_per_household = (
    df_housing.groupby('neighborhood')['bedrooms_per_household']
    .mean()
    .reset_index()
)

# Rename the column for clarity
avg_bedrooms_per_household.columns = ['neighborhood', 'bedrooms-per-household']

# Merge into df_neighborhood
df_neighborhood = df_neighborhood.merge(avg_bedrooms_per_household, on='neighborhood', how='left')

# Save the modified DataFrame
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Round the column to the nearest integer
df_neighborhood['bedrooms-per-household'] = df_neighborhood['bedrooms-per-household'].round(0).astype('Int64')

# Save the modified DataFrame to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# One-hot encode the 'ocean_proximity' column
df_housing = pd.get_dummies(df_housing, columns=['ocean_proximity'])

# Save the expanded DataFrame to a CSV file
df_housing.to_csv('housing_ocean_proximity_encoded.csv', index=False)

# Display the result
print(df_housing.head())

# Save the updated DataFrame
df_housing.to_csv('df_housing.csv', index=False)

In [ ]:
# Group df_housing to ensure one row per neighborhood, taking the first occurrence or mode
df_ocean = df_housing[['neighborhood', 'ocean_proximity_<1H OCEAN']].dropna().drop_duplicates(subset='neighborhood')

# Merge with df_neighborhood on the 'neighborhood' column
df_neighborhood = df_neighborhood.merge(df_ocean, on='neighborhood', how='left')

# Save to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Group df_housing to ensure one row per neighborhood, taking the first occurrence or mode
df_ocean = df_housing[['neighborhood', 'ocean_proximity_INLAND']].dropna().drop_duplicates(subset='neighborhood')

# Merge with df_neighborhood on the 'neighborhood' column
df_neighborhood = df_neighborhood.merge(df_ocean, on='neighborhood', how='left')

# Save to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Group df_housing to ensure one row per neighborhood, taking the first occurrence or mode
df_ocean = df_housing[['neighborhood', 'ocean_proximity_ISLAND']].dropna().drop_duplicates(subset='neighborhood')

# Merge with df_neighborhood on the 'neighborhood' column
df_neighborhood = df_neighborhood.merge(df_ocean, on='neighborhood', how='left')

# Save to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Group df_housing to ensure one row per neighborhood, taking the first occurrence or mode
df_ocean = df_housing[['neighborhood', 'ocean_proximity_NEAR BAY']].dropna().drop_duplicates(subset='neighborhood')

# Merge with df_neighborhood on the 'neighborhood' column
df_neighborhood = df_neighborhood.merge(df_ocean, on='neighborhood', how='left')

# Save to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
# Group df_housing to ensure one row per neighborhood, taking the first occurrence or mode
df_ocean = df_housing[['neighborhood', 'ocean_proximity_NEAR OCEAN']].dropna().drop_duplicates(subset='neighborhood')

# Merge with df_neighborhood on the 'neighborhood' column
df_neighborhood = df_neighborhood.merge(df_ocean, on='neighborhood', how='left')

# Save to CSV
df_neighborhood.to_csv('df_neighborhood.csv', index=False)

In [ ]:
df_neighborhood.head(10)

In [ ]:
# Define the neighborhoods of interest
neighborhoods_of_interest = ["Brooktree", "Fisherman's Wharf", "Los Osos"]

# Filter and display the matching rows
filtered_df = df_neighborhood[df_neighborhood['neighborhood'].isin(neighborhoods_of_interest)]
print(filtered_df)

# ----------

In [ ]:
identity_data_object = s3_client.get_object(
    Bucket=fraud_detection_bucket_name, Key=identity_file_key
)
transaction_data_object = s3_client.get_object(
    Bucket=fraud_detection_bucket_name, Key=transaction_file_key
)

identity_data = pd.read_csv(io.BytesIO(identity_data_object["Body"].read()))
transaction_data = pd.read_csv(io.BytesIO(transaction_data_object["Body"].read()))

In [ ]:
# Inspect Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io

s3_client = boto3.client("s3", region_name=region)

#XXXXXXXXXXXXXXXXXX
#fraud_detection_bucket_name replaced with home_value_prediction

home_value_prediction = f"sagemaker-example-files-prod-{region}"
identity_file_key = ("housing_gmaps_data_raw.csv"
    #"datasets/tabular/fraud_detection/synthethic_fraud_detection_SA/sampled_identity.csv"
)
transaction_file_key = ("housing.csv"
    #"datasets/tabular/fraud_detection/synthethic_fraud_detection_SA/sampled_transactions.csv"
)

identity_data_object = s3_client.get_object(
    Bucket=home_value_prediction, Key=identity_file_key
)
transaction_data_object = s3_client.get_object(
    Bucket=home_value_prediction, Key=transaction_file_key
)

identity_data = pd.read_csv(io.BytesIO(identity_data_object["Body"].read()))
transaction_data = pd.read_csv(io.BytesIO(transaction_data_object["Body"].read()))


In [ ]:
identity_data.head()

In [ ]:
transaction_data.head()